In [ ]:
import os
import sys
import fnmatch
import zipfile
import xmltodict
import numpy as np
import pandas as pd
import json
import gzip
import pickle
import csv
import scipy.sparse

In [ ]:
# nsf data
df2 = pd.read_pickle('nsf2.pkl')
df = pd.read_pickle('nsf.pkl')
Xauth = None

In [ ]:
r1_confs = pickle.load(open('r1_confs.pkl','rb'))
r1_confs_dict = {_:1 for _ in r1_confs}

In [ ]:
# from the big paper thing
papers = pd.read_hdf('papers.h5','table')
unique_names = pickle.load(open('big_names.pkl','rb'))
unique_confs = pickle.load(open('confs.pkl','rb'))

faculty_affil = pd.read_csv('faculty-affiliations.csv')
ranks = pd.read_csv('ranks.csv')
def csv2dict_str_str(fname):
    with open(fname, mode='r') as infile:
        rdr = csv.reader(infile)
        d = {rows[0].strip(): rows[1].strip() for rows in rdr}
    return d
aliasdict = csv2dict_str_str('dblp-aliases.csv')
conf_idx = pickle.load(open('conf_idx.pkl','rb'))
name_idx = pickle.load(open('name_idx.pkl','rb'))

In [ ]:
# munge the years
min_year = df.year.min()
max_year = df.year.max()
span_years = max_year - min_year
year_blocks = 8 #11
offset_years = [(i-min_year)//year_blocks for i in range(min_year,max_year+1)]
year_ind = max(offset_years)+1
print(min_year,max_year)

In [ ]:
nsf_paper_n, _ = df2.shape
nsf_paper_n
offset_years,year_ind

In [ ]:
from collections import Counter,defaultdict
# create or load some author data
def dd():
    return defaultdict(list)
if False:
 
    author_papers = defaultdict(dd)

    for row in papers.itertuples():
        paper_year = row[10]
        conf = row[2]
        n = row[4]
        authors = row[3]
        for a in authors:
            split_name = a.split(' ')
            first_last = split_name[0] +' ' + split_name[-1]
            author_papers[first_last][paper_year].append((conf,n))
    import pickle
    with open('nsf_auth.pkl','wb') as fp:
        pickle.dump(author_papers,fp)
else:
    pass
    with open('nsf_auth.pkl','rb') as fp:
        author_papers = pickle.load(fp)

In [ ]:
def ddn():
    return defaultdict(int)
author_amounts = defaultdict(ddn)
for i,row in enumerate(df2.itertuples()):
    authors, year, amount = row[3],row[4],row[5]
    # some infinite amounts exist! bad!
    if not np.isfinite(amount):
        continue
    for a in authors:
        split_name = a.split(' ')
        first_last = split_name[0] +' ' + split_name[-1]
        for yr in range(int(year),2020):
            author_amounts[first_last][yr] += amount/len(a)#min(amount/len(a),1e6)

y = np.zeros(nsf_paper_n,dtype=np.float32)
for i,row in enumerate(df2.itertuples()):
    authors, year, amount = row[3],row[4],row[5]
    # some infinite amounts exist! bad!
    
    #if not np.isfinite(amount):
    #    continue

    #if amount < 1000: #50000
    #    continue
    # giant grants are msileading?
    #if amount >= 1e9: #5e7
    #    continue
    if len(authors)  <= 0.5*sum([(a in author_papers) for a in authors]):
        y[i] = amount#sum([author_amounts[first_last][year] for first_last in authors]) #or amount!?

y_orig = np.copy(y)


In [ ]:
if True:
    y = np.zeros(nsf_paper_n,dtype=np.float32)
    for i,row in enumerate(df2.itertuples()):
        authors, year, amount = row[3],row[4],row[5]
        # some infinite amounts exist! bad!
        if not np.isfinite(amount):
            continue

        if amount < 1000: #50000
            continue
        # giant grants are msileading?
        if amount >= 1e8: #5e7
            continue
        if len(authors) < 1:
            continue
        if len(authors)*0.5  <= sum([(a in author_papers) for a in authors]):
            y[i] = amount/len(authors)#sum([author_amounts[first_last][year] for first_last in authors]) #or amount!?
import matplotlib.pyplot as plt
y_orig = np.copy(y)
_ = plt.hist(y,100)

In [ ]:
# create design mattrix
X = scipy.sparse.dok_matrix((nsf_paper_n,year_ind*unique_confs.shape[0]))
xdict = {}
y = np.zeros(nsf_paper_n,dtype=np.float32)
for i,row in enumerate(df2.itertuples()):
    authors, year, amount = row[3],row[4],row[5]
    # some infinite amounts exist! bad!
    if not np.isfinite(amount):
        continue
    # maybe the old years are misleading!?
    #if year < 2002:
    #    continue
    # small grants are misleading? 150000
    if amount < 50000:
        continue
    # giant grants are msileading?
    if amount >= 5e7:
        continue
    #print(len(authors),sum([(a in author_papers) for a in authors]))
    #print(a)
    if len(authors)*0.5 <= sum([(a in author_papers) for a in authors]):
        y[i] = amount
        for a in authors:
            for year_a,conf_list in author_papers[a].items():
                if year_a <= year:
                    for paper in conf_list:
                        #if paper[0] in noisey_crap:
                        #    continue
                        if paper[0] == 'CoRR':
                            continue
                        if paper[0] not in r1_confs_dict:
                            continue
                        j = year_ind*conf_idx[paper[0]] + (year_a-min_year)//year_blocks
                        xdict[(i,j)] = 1/paper[1]
X._update(xdict)
y_orig = np.copy(y)

In [ ]:
if False:
    X = scipy.sparse.lil_matrix(X.T)
    for i in range(len(unique_confs)):
        if unique_confs[i] not in r1_confs_dict:
            X[i,:] = 0
    X = scipy.sparse.csr_matrix(X.T)

In [ ]:
#y_orig = np.copy(y)
print(y_orig.min(),y_orig.max())
print((y_orig > 0).sum())
y = np.copy(np.log(1+y_orig))
#y = np.copy(y_orig)
#y[y > 1e6] = 1e6
y[y == np.log(1)] = y[y != np.log(1)].mean()
#y[y == 0] = y[y != 0].mean()

from matplotlib.pyplot import figure,hist
hist((y-y.mean())/y.std(),100)
figure()
_ = hist(y,100)

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR

X = scipy.sparse.csr_matrix(X)
clf = SGDRegressor('huber',tol=1e-9,max_iter=100,verbose=0,penalty='l2',alpha=1e-7,epsilon=0.01,average=True)
#clf = SGDRegressor('huber',tol=1e-9,max_iter=100,verbose=1,penalty='l1',alpha=1e-7)

clf.fit(X,(y-y.mean())/y.std())

In [ ]:
conf_ord = np.argsort(np.squeeze(clf.coef_))
conf_choice = ['SIGGRAPH','HRI','ECCV (8)','ECCV (1)','Comput. Graph. Forum','Shape Modeling International','Symposium on Geometry Processing','Computer Aided Geometric Design','I. J. Robotics Res.','CVPR','International Journal of Computer Vision','Robotics: Science and Systems','ICRA','WACV','ICML','AISTATS','CoRR','SIGGRAPH Asia','ECCV','ICCV','ISER','Humanoids','3DV','IROS','CoRL','Canadian Conference on AI','ACCV','Graphics Interface','CRV','BMVC']
ri_confs = np.zeros(len(unique_confs)*year_ind)
print(clf.intercept_)
ms = clf.coef_.mean()
ss = clf.coef_.std()
for i in range(len(unique_confs)*year_ind):
    idx = conf_ord[-(i+1)]
    conf_name = unique_confs[idx//year_ind]
    conf_score = clf.coef_[idx]
    if conf_name in conf_choice:
        ri_confs[idx] = 1
    if conf_name in conf_choice and (idx%year_ind)==5:
    #if 'ICCV' in conf_name and (idx%year_ind)==4:
        start_year = offset_years.index(idx%year_ind) + 1970
        end_year = len(offset_years) - 1 - offset_years[::-1].index(idx%year_ind) + 1970
        print_name =conf_name + '_' + str(start_year)[-2:] +'t' + str(end_year)[-2:]
        print('{:20s}\t{:.3f}\t{:.1f}'.format(print_name[:20],100*conf_score,(conf_score-ms)/ss))
ri_confs.shape,ri_confs.sum(),X.shape

In [ ]:
top_k = 50
for i in range(top_k):
    idx = conf_ord[-(i+1)]
    conf_name = unique_confs[idx//year_ind]
    conf_score = clf.coef_[idx]
    start_year = offset_years.index(idx%year_ind) + 1970
    end_year = len(offset_years) - 1 - offset_years[::-1].index(idx%year_ind) + 1970
    print_name = conf_name[:10] + '_' + str(start_year)[-2:] +'t' + str(end_year)[-2:]
    print('{:20s}\t\t\t\t{:.3f}\t{:.2f}'.format(print_name,100*conf_score,(conf_score-ms)/ss))

In [ ]:
hist(clf.coef_,70)
print((clf.coef_ > 0.15).sum())
noisey_crap2 = set([unique_confs[i//year_ind] for i,_ in enumerate(clf.coef_ > 0.15) if _])

In [ ]:
x1 = X.T.sum(1)
noisey_crap2


In [ ]:
#noisey_crap = noisey_crap.union(noisey_crap2)

In [ ]:
#noisey_crap

In [ ]:
if True: #or Xauth is None or (Xauth.shape[1] != year_ind*unique_confs.shape[0]):  
    Xauth = scipy.sparse.dok_matrix((len(unique_names),year_ind*unique_confs.shape[0]))
    xdict = {}
    auth_years = np.ones((len(unique_names),2)) * np.array([3000,1000]) 
    for row in papers.itertuples():
        paper_year = row[10]
        #if row['year'] < 2005:
        #    continue
        #print(row)
        #if row['conf'] == 'CoRR':
        #    continue
        conf = row[2]
        n = row[4]
        authors = row[3]
        j = year_ind*conf_idx[conf] + (paper_year-min_year)//year_blocks
        for a in authors:
            i = name_idx[a]
            xdict[(i,j)] = 1/n + xdict.get((i,j),0)
            auth_years[i,0] = min(auth_years[i,0],paper_year)
            auth_years[i,1] = max(auth_years[i,1],paper_year)
    Xauth._update(xdict)

In [ ]:
scores = clf.predict(Xauth) - np.squeeze(clf.intercept_)
years_working = (1+auth_years[:,1]-auth_years[:,0])
value_scores = scores
norm_scores = (value_scores)/years_working
ri_filter_mat = scipy.sparse.diags(ri_confs)
ri_scores = clf.predict(Xauth.dot(ri_filter_mat))-np.squeeze(clf.intercept_)
ri_norm_scores = ri_scores/years_working

In [ ]:
prev_cand = ['Pulkit Agrawal',
 'Joydeep Biswas',
 'Katherine L. Bouman',
 'David Braun',
 'Jia Deng',
 'Naomi T. Fitter',
 'David F. Fouhey',
 'Saurabh Gupta',
 'Judy Hoffman',
 'Hanbyul Joo',
 'Honglak Lee',
 'Changliu Liu',
 'Petter Nilsson',
 "Matthew O'Toole",
 'Alessandro Roncone',
 'Alanson P. Sample',
 'Manolis Savva',
 'Adriana Schulz',
 'Amy Tabb',
 'Fatma Zeynep Temel',
 'Long Wang',
 'Cathy Wu',
 'Ling-Qi Yan']
print('{:20s}\t{:4s}\t{:4s}\t{:4s}\t{}'.format('name','rate','total','ri','years'))
for ns, name in sorted([(value_scores[name_idx[ni]],ni) for ni in prev_cand],reverse=True):
    ni = name_idx[name]
    print('{:20s}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.0f}'.format(name,100*norm_scores[ni],100*value_scores[ni],100*ri_scores[ni],years_working[ni]))
print('')
curious_names = ['Xiaolong Wang 0004','Judy Hoffman','Paris Siminelakis',
                 'Nicholas Rhinehart',
                 'Humphrey Hu',
                 'David F. Fouhey',
                 'Lerrel Pinto',
                 'Justin Johnson',
                 'Amir Roshan Zamir',
                 'Brian Okorn','David Held']
print('{:20s}\t{:4s}\t{:4s}\t{:4s}\t{}'.format('name','rate','total','ri','years'))
for _,name in sorted([(value_scores[name_idx[_]],_) for _ in curious_names],reverse=True):
    ni = name_idx[name]
    print('{:20s}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.0f}'.format(name,100*norm_scores[ni],100*value_scores[ni],100*ri_scores[ni],years_working[ni]))

In [ ]:
print('\n best overall \n')
cmu_scores = []

best_scores = np.argsort(value_scores)[::-1]
#print(best_scores.shape,unique_names[best_scores[0]])
fa_list = list(faculty_affil.name)
fa_a_list = list(faculty_affil.affiliation)
uni_names = [unique_names[i] for i in best_scores[:38000]]
for name in set([aliasdict.get(n, n) for n in uni_names]):
    if name in name_idx:
        uni = 'unknown'
        if name in fa_list:
            uni = fa_a_list[fa_list.index(name)]
        if name not in []:#['Jacob Walker','Justin Johnson','Pieter Abbeel','Martial Hebert','Jessica K. Hodgins','Abhinav Gupta','Christopher G. Atkeson','Tom M. Mitchell','Matthew T. Mason']:
            if years_working[name_idx[name]] < 3:
                continue
            if years_working[name_idx[name]] > 8:
                continue
            if ri_scores[name_idx[name]] < 0.008:
                continue
            if auth_years[name_idx[name],1] < 2016:
                continue
        #if (np.array(X[name_idx[name],:].todense()) * ri_confs).sum() == 0:
        #    continue
        #print(name,auth_years[name_idx[name]])
        score = norm_scores[name_idx[name]]
        ri_vscore = ri_norm_scores[name_idx[name]]
        vscore = value_scores[name_idx[name]]
        cmu_scores.append((vscore,ri_scores[name_idx[name]],score,uni,name,auth_years[name_idx[name]],ri_vscore))
    else:
        pass
        #print(name)
        ri_norm_scores
print('{:22s}\t{:15s}\t{:5s}\t{:3s}\t{:4s}\t{:4s}\t{} {}'.format('name','uni','rate','RI-t','total','RI-r','start','end'))
for vs,ris,s,u,p,yrs,rir in sorted(cmu_scores,reverse=True):
    print('{:22s}\t{:15s}\t{:.3f}\t{:.1f}\t{:.2f}\t{:.2f}\t{} {}'.format(p[:22],u[:15],s*100,ris*100,vs*100,rir*100,int(yrs[0]),int(yrs[1])))


In [ ]:
uni_faculty = faculty_affil[faculty_affil.affiliation == 'Carnegie Mellon University'] #Carnegie Mellon University
uni_names = np.array(uni_faculty.name)
uni_names = list(uni_names) + ['Nicholas Rhinehart','Jacob Walker','Lerrel Pinto','Brian Okorn','Leonid Keselman','Siddharth Ancha','Humphrey Hu']
cmu_scores = []
#uni_names = [unique_names[i] for i in (np.argsort(scores)[::-1])[:150]]
for name in set([aliasdict.get(n, n) for n in uni_names]):
    if name in name_idx:
        #if ri_scores[name_idx[name]] < 2.5:
        #    continue
        score = scores[name_idx[name]]
        cmu_scores.append((score,name))
    else:
        pass
        #print(name)
for s,p in sorted(cmu_scores,reverse=True):
    print('{:30s}\t\t{:.3f}'.format(p,s*100))


In [ ]:
pickle.dump(clf.coef_,open('clf_nsf_ind_log_r1.pkl','wb'))


In [ ]:
pickle.dump(Xauth,open('xauth.pkl','wb'))


In [ ]:
du -h *.pkl